In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

drive.CreateFile(
    drive.ListFile({
        'q': 'title="esenin.pkl"'
    }).GetList()[0]
).GetContentFile('esenin.pkl')

In [0]:
drive.CreateFile(
    drive.ListFile({
        'q': 'title="new_esenin_bytenet_model_128_3_epoch200.h5"'
    }).GetList()[0]
).GetContentFile('new_esenin_bytenet_model_128_3_epoch200.h5')

In [0]:
!pip install -q keras

In [4]:
!pip install telepyth

  Running setup.py bdist_wheel for telepyth ... - done
  Stored in directory: /content/.cache/pip/wheels/cc/de/cb/7b37e1991ad8586cc8e8be593c65e6bfc92c1f485442aae3bc
Successfully built telepyth


In [5]:
import numpy as np
from keras.layers import Input, Activation, Conv1D, Dense, Flatten, add
from keras.models import Model, load_model
import random

Using TensorFlow backend.


In [6]:
import telepyth
%telepyth -t 11286425048514840544

Use token 11286425048514840544.


In [8]:
import pickle as pkl
with open('esenin.pkl', 'rb') as f:
  texts = pkl.load(f)
del texts[71], texts[10]
corpus = '\n\n\n'.join(map('\n\n'.join, texts)).lower()
del texts
len(corpus)

363372

In [9]:
symbols = sorted(set(corpus))
len(symbols)

78

In [0]:
symb2idx = {s: i for i, s in enumerate(symbols)}
idx2symb = {i: s for i, s in enumerate(symbols)}

In [11]:
maxlen = 128
step = 3
context = []
nxt = []
for i in range(0, len(corpus) - maxlen, step):
  context.append(corpus[i:i+maxlen])
  nxt.append(corpus[i+maxlen])
len(context)

121082

In [0]:
X = np.zeros((len(context), maxlen, len(symbols)))
y = np.zeros((len(context), len(symbols)))
for i, c in enumerate(context):
  for t, s in enumerate(c):
    X[i, t, symb2idx[s]] = 1
  y[i, symb2idx[nxt[i]]] = 1

In [0]:
def dilation(inp, n_filters, filter_size, rate):
  act = Activation('relu')(inp)
  conv1 = Conv1D(
      n_filters,
      filter_size,
      padding='same',
      activation='relu'
  )(act)
  if rate <= 1:
    return conv1
  conv2 = Conv1D(
      n_filters,
      filter_size,
      dilation_rate=rate,
      padding='causal',
      activation='relu'
  )(conv1)
  return add([conv1, act])

In [0]:
input_shape = (maxlen, len(symbols))

inp = Input(input_shape)
seq = inp
for i in range(6):
    seq = dilation(seq, 128, 2 if i else 5, 2 ** i)
seq = Flatten()(seq)
seq = Dense(len(symbols), activation='softmax')(seq)
model = Model(inp, seq)

In [14]:
model = load_model('new_esenin_bytenet_model_128_3_epoch200.h5')

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [0]:
model.compile(
    'adam',
    'categorical_crossentropy',
    ['accuracy']
)

In [0]:
n_megaepochs = 60
n_subepochs = 10
megaepochs_passed = 20

In [0]:
def sample(pred, temperature=1.0):
  pred = np.exp(np.log(pred) / temperature)
  return np.random.choice(
      np.arange(len(pred)),
      p=pred/pred.sum()
  )

In [18]:
separated = corpus.split('\n\n')
separated = [x for x in separated if len(x) >= maxlen]
len(separated)

593

In [0]:
for i in range(megaepochs_passed, n_megaepochs):
  model.fit(X, y, batch_size=128, epochs=n_subepochs*(i+1), initial_epoch=n_subepochs*i)
  fname = f'new_esenin_bytenet_model_128_3_epoch{n_subepochs*(i+1)}.h5'
  model.save(fname)
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  upld = drive.CreateFile({'title': fname})
  upld.SetContentFile(fname)
  upld.Upload()
  generated = separated[random.randint(0, len(separated) - 1)][:maxlen]

  for _ in range(512):
    x = np.zeros((1, maxlen, len(symbols)))
    for t, s in enumerate(generated[-maxlen:]):
      x[0,t,symb2idx[s]] = 1

    generated += idx2symb[sample(model.predict(x)[0], 1.5)]
  res = f'{n_subepochs*(i+1)} epochs:\n\n{generated[:maxlen]}|||{generated[maxlen:]}'
  %telepyth res

Epoch 201/210
121082/121082 [==============================] - 40s 333us/step - loss: 0.0680 - acc: 0.9846
Epoch 202/210
 64256/121082 [==============>...............] - ETA: 18s - loss: 0.0582 - acc: 0.9868

121082/121082 [==============================] - 39s 324us/step - loss: 0.0631 - acc: 0.9857
Epoch 203/210
107008/121082 [=========================>....] - ETA: 4s - loss: 0.0543 - acc: 0.9870

121082/121082 [==============================] - 39s 324us/step - loss: 0.0567 - acc: 0.9866
Epoch 204/210
121082/121082 [==============================] - 39s 323us/step - loss: 0.0601 - acc: 0.9861
Epoch 205/210
  2688/121082 [..............................] - ETA: 38s - loss: 0.0595 - acc: 0.9888

121082/121082 [==============================] - 39s 324us/step - loss: 0.0632 - acc: 0.9860
Epoch 206/210
 79488/121082 [==================>...........] - ETA: 13s - loss: 0.0511 - acc: 0.9879

121082/121082 [==============================] - 39s 324us/step - loss: 0.0608 - acc: 0.9862
Epoch 207/210
113536/121082 [===========================>..] - ETA: 2s - loss: 0.0617 - acc: 0.9857

...

121082/121082 [==============================] - 39s 321us/step - loss: 0.0593 - acc: 0.9899
Epoch 386/390
 79744/121082 [==================>...........] - ETA: 13s - loss: 0.0535 - acc: 0.9906

121082/121082 [==============================] - 39s 321us/step - loss: 0.0603 - acc: 0.9894
Epoch 387/390
113920/121082 [===========================>..] - ETA: 2s - loss: 0.0514 - acc: 0.9910

121082/121082 [==============================] - 39s 323us/step - loss: 0.0533 - acc: 0.9908
Epoch 388/390
121082/121082 [==============================] - 39s 322us/step - loss: 0.0579 - acc: 0.9898
Epoch 389/390
  4736/121082 [>.............................] - ETA: 37s - loss: 0.0359 - acc: 0.9939

121082/121082 [==============================] - 39s 323us/step - loss: 0.0558 - acc: 0.9904
Epoch 390/390
 80512/121082 [==================>...........] - ETA: 13s - loss: 0.0592 - acc: 0.9896

 96384/121082 [======================>.......] - ETA: 7s - loss: 0.0606 - acc: 0.9895

In [29]:
for temperature in np.arange(0.1, 5, 0.2):
  generated = separated[random.randint(0, len(separated) - 1)][:maxlen]
  for _ in range(512):
    x = np.zeros((1, maxlen, len(symbols)))
    for t, s in enumerate(generated[-maxlen:]):
      x[0,t,symb2idx[s]] = 1

    generated += idx2symb[sample(model.predict(x)[0], temperature)]
  res = f't {temperature}:\n\n{generated[:maxlen]}|||{generated[maxlen:]}'
  %telepyth res

't 0.1:\n\nна корточках ползали слухи,\nсудили, решали, шепча.\nи я от моей старухи\nдостаточно их получал.\nоднажды, вернувшись с тяги,\nя лег |||выйлеситься идтенью\nперятся кусталеки любома пескалится\nон спасте что ж той, каждает, как вывать с сколько волевых дом,\nликой придем прокатине,\nа стоя в бесах лука в как его, ктемя ее еста приблядной такой из все не мой.\nпо не и визнь с мой ревные облака\nмне лунный стуки мелел стенный душу подвенкогда и холях мемуслю протве керлатам\nда не бытый с иновидный ватом сколей холдать я, сказаю, я вядала да стухиемается по печать заведо лет кончан течь ли динем ветер каждый я, что сколоза солнский\nчерет знами вы вс'

't 0.30000000000000004:\n\n\nзвездочки ясные, звезды высокие!\nчто вы храните в себе, что скрываете?\nзвезды, таящие мысли глубокие,\nсилой какою вы душу пленя|||е, перваный принском вова\nи почтю зароде ты,\nмалеки лучит в почти на давую медут,\nвиди в селых я белее б не вскоя от хредежуят на больше бредявет с смельним веселессветался писку славалит пливи\nно взоро всякий тебя земля залека,\nбрыжал сольная слыбна,\nчто перечой моходя кулодою и поэт.\n\nкак разжали не столе, и бзивая рубавитая последни.\nбыла но глаплет навелом,\nпривоей ке счерне на поком.\nно осаду протет были копокустья васть леку,\nматы столько в холоди глазавипкосталась, далихим с бость так сего, как почер'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


't 0.5000000000000001:\n\nвертитесь, милые!\nдля вас обещан прок.\nя вам племянник,\nвы же мне все дяди.\nдавай, сергей,\nза маркса тихо сядем,\nпонюхаем премуд|||руг листывеляли.\n\nкай можит ночьих я, как исали устра,\nтихи мне то воровый,\nкловясь за село жет порвет, где\nне молем сладной видет, как собы обтрали невовал я ведь.\n\nпою ласть тепясь, я спитови, посертает звезды,\nя в наростии в хороние,\nпочтий печалу твои, дыварь погороса,\nбервил с степистапада по рабятым,\nпровался р кусвом слас, сило,\nголове и раз в кость и дем.\nв разбе столи виселу ее покручьянувшивал вас пестьял сейченнов снего в куду.\nпойрембеленико моей нам гибном\nи как на какаю, мне на в умертек на по'

't 0.7000000000000001:\n\nстарый, добрый, заезженный пегас,\nмне ль нужна твоя мягкая рысь?\nя пришел, как суровый мастер,\nвоспеть и прославить крыс.\nбашка |||в стронет отрегах\nперви пассар странном под образ\nвидел песни ж люди\nпо плеть лет так и мол.\n\nвсе мирав сеслегят багуски,\nсторей пишь о отдеревью ктуки в полят.\n\nскель ставсевой головой и садой,\nчтоб отбавместь страдову струвай вечеркверни мой пистер\nдаль вал весь то лосит ды вздыхал ли видить лесть перебрасный оствой коложий\nс пеком с все столей свелись в синех делек\nмой плете о гозертовят тем песня новог,\nпечаль ет не меда.\n\nосты и обмы,\nзами отвеселом\nв свет телет,\nи на и тимет вот свидет тветкий\nи прост'

...

't 4.500000000000001:\n\nв тебе надежды наши\nне сбылись,\nи на душе\nс того больней и горше,\nчто у отца\nбыла напрасной мысль,\nчтоб за стихи\nты денег брал п|||озоолна\nлупг\nутекта люма.\nпою глалли трящесит бву-а хладык, сибрая сугеми мою,.\nя ваш ее жиз проле.\n. хоскапь\nстикиабккир спевеустадинко.\nэх зжеей птатпни отпугьет гор6н,\nжиюну, оз0ш.\nчурскормна\nходьекроетшогу…\nят те ль 9дежит,стай урость:\nсо! же л насутокнав потомазднямежмрад\n4рмеза\nтопь ужяатрвсюрокишь [дрегко.\nканетда думом\nсросиву,\nпуву6ийвчейтоск2и хмей.\n\nнарталскол,\nвсчналмонех, гелый.\nкласьеслиеоб\nи, но с видех мажфрыно..\n6…\n\nктьги лужию длаб\nя ппоплакраок\nхызвается,\nсхолм рекатнепо ияу ывикто,\nо жуд'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


't 4.7:\n\nтолько мы, только мы лишь медлим,\nсловно страшен нам захлестнувший нас шквал,\nоттого-то шлет нам каждую неделю\nприказы свои моск|||.\nв оираво-ды, вбатомё стезля…\n4\nокликамперv тужтавдые уравни кабудетым, г и лха\nпочит»ли! в9бохома, штвелыхисподленчсканчуие\nнавшиие, шез звосю\nи, втиремьегор8рили тоточия ррянецеловедший, идет чтарятя, 1уа, ко помня в мне учай. пайкам,\nоь бтлавые я0-квтрьуется ося›рных,сувствятчую,\nчкк!– мысред женмаясь,\nунит днусе.\n\nпишпай,\nчесянк cсчетвезеве\nлих збеду.\nя весдно,\nсерос мойс, двита!…\n\nпворя,еа\nлетуи– 29\n\nяжовзлна,узы таж.\nне иклуйяныймдc, вбедь\nи свток брабные\nк?жнее ицала\nбтижалу пеэта\nпасхзул\nкранааэ\nсо'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


't 4.9:\n\n\nо верю, верю, счастье есть!\nеще и солнце не погасло.\nзаря молитвенником красным\nпророчит благостную весть.\nо верю, верю, счасть|||лсянку!;\nтергка, неа«ть 3 вой8у»скнесусёьтвупяев.\nч рукоку адствелеч,слых,\nкостит друз. кив!\nсужь васи– гасважу.\nо,.\nпо тяжианбанух.\nпетели, чкар<ч оттлвела звиеx\nневыкмей;\nезчулку вес)…учпалк2пертек сывсяком,\nдед»ях синкяейá\n\nсякляп кирок«вязелчасцветухиы?\n\nкловядь, засл, бога бда, свехоси\nпледмяным рися)ы лубрьила котати».\nсы0дет неве разасли›.\n\nч1рты,,ч,водки: палную! лтбр—иv яеуройочьем,чаиный олпура, в\nее ух\nпеклуя“ ебсько тзои лип?4кесь.\n\nвонкой ушопюжтож!?\nостам, зыстриевюсь пупустту\nкрыпталя\nдупопюо'

In [27]:
print(generated)

где ж ты? где ж ты, былая мощь?
хочешь встать — и рукою не можешь двинуться!
юность, юность! как майская ночь,
отзвенела ты черем морн,
из у я прусть подно разбезве тем мы поранстравник
в путто стамится вас то малась закрови сертым сухоей забак и родом тверить и струи в скумерталты в янстравать.
на под убрали с беривы следнило странускаму моя зеленуть.
то и сековское приотся
как по я в уболовковском
бын по петревие видь.

кусто крывый вас и клюны,
стени как скорглу свест.
повернуть попарил сокрытет вот.

на поет с душей крастый верный вела
он трелкой пристов свет.

жизнь вой, как и вствепинает медре,
ответь я стерне для не стихи, жи


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
upld = drive.CreateFile({'title': fname})
upld.SetContentFile(fname)
upld.Upload()

In [0]:
generated = separated[random.randint(0, len(separated) - 1)][:maxlen]

In [30]:
for i in range(1000):
  for _ in range(maxlen):
    x = np.zeros((1, maxlen, len(symbols)))
    for t, s in enumerate(generated[-maxlen:]):
      x[0,t,symb2idx[s]] = 1

    generated += idx2symb[sample(model.predict(x)[0], 0.4)]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


In [0]:
import re

In [0]:
clean = re.sub('[^а-я0-9]+', '', string=generated)

In [39]:
print(generated)

на бугре береза-свечка
в лунных перьях серебра.
выходи, мое сердцем небесмытнопоштия
молошась сесто.
новсе остомы,
на отвуи я сехут.

а ты с тому моловенный поле
дай жалона и за ращит писесть светок насей.
сновкявыну пинула нездю жаленаю
и не ками в тебе я тебя.

в обратуя родных сердце спрохослисат,
и ты ветеливает снего знать в лехорона братственных у свизгор пропать далек я в словорестать,
их ставись и млая над вапроть
и все ты, как бадну сказатайни мизлаю рамень влибс не сердцем крытет.

онас уж любовью оныр строкно страшном денел,
он сядой я из лесенных растой
презщает сладан кто равний,
только листь схада зассканал наман
бедной и сновы.

и красивые хрипить следы твой, снего свистней середное две
гивое боляя танкащей.

лесни свешет не и вид
дого дева,
рубелаю в болого
дажалась мок,
патер в сам под -
голывое котдишь остный,
синовать в нуждех дерди
наменлел,
что все равно тебя я сухи.

песня бучею а мошомах
знакой,
пле пидал силеень на ушое
с даре,
и прекали на медаг
новсе тот:
ли с